In [41]:
import numpy as np
from collections import defaultdict
from scipy.spatial import distance

In [6]:
A = defaultdict(lambda: 0)
B = defaultdict(lambda: 0)
A1 = {1:3, 3:4, 5:5}
B1 = {1:8, 2:9, 5:10}
A.update(A1)
B.update(B1)
a1, b1 = list(), list()

In [7]:
for idx, tag in enumerate(list(set(A.keys()).union(set(B.keys())))):
    a1.append(A[tag])
    b1.append(B[tag])

In [8]:
a1

[3, 0, 4, 5]

In [9]:
list(set(A.keys()).union(set(B.keys())))

[1, 2, 3, 5]

In [28]:
with open("tag_distributions/tag_dist_l.json", "r") as l_file:
    tag_dist_l = json.load(l_file)
    
with open("tag_distributions/tag_dist_h.json", "r") as h_file:
    tag_dist_h = json.load(h_file)

In [29]:
def find_kl_cost(dist_l, dist_h):
    A = defaultdict(lambda: 0)
    B = defaultdict(lambda: 0)
    A.update(dist_l)
    B.update(dist_h)
    list_l, list_h = list(), list()
    for idx, tag in enumerate(list(set(A.keys()).union(set(B.keys())))):
        list_l.append(A[tag])
        list_h.append(B[tag])
    
    epsilon = 0.0001
    list_l = np.asarray(list_l) + epsilon
    list_l = list_l/list_l.sum() 
    list_h = np.asarray(list_h) + epsilon
    list_h = list_h/list_h.sum() 

    kld = list_l*np.log(list_l/list_h)
    
    return kld.sum()


In [83]:
import time
def find_cosine_cost(dist_l, dist_h):
    start = time.time()
    A = defaultdict(lambda: 0)
    B = defaultdict(lambda: 0)
    A.update(dist_l)
    B.update(dist_h)
    list_l, list_h = list(), list()
    for idx, tag in enumerate(list(set(A.keys()).union(set(B.keys())))):
        list_l.append(A[tag])
        list_h.append(B[tag])
    
    end = time.time()
#     print("LISTING DICT: ", end-start)
    start = time.time()
#     cos = distance.cosine(list_l, list_h)
    cos = np.dot(np.asarray(list_l), np.asarray(list_h))
    end = time.time()

#     print("COSINE: ", end-start)
    return cos



In [86]:
def populate_cosine_cost_array(tag_dist_l, tag_dist_h):
    A = np.zeros([len(tag_dist_l), len(tag_dist_h)])
    for i_l, m_l in enumerate(list(tag_dist_l.keys())):
        if i_l % 1000 == 0:
            print(i_l)
        for i_h, m_h in enumerate((tag_dist_h.keys())):
            A[i_l][i_h] = find_cosine_cost(tag_dist_l[m_l], tag_dist_h[m_h])
    return A

In [ ]:
cosine_cost_array = populate_cosine_cost_array(tag_dist_l2, tag_dist_h2)

0
1000
2000


In [70]:
tag_dist_h2 = {}
for k, dist in list(tag_dist_h.items()):
    top2 = sorted(list(dist.items()),key= lambda x: x[1], reverse=True)[:2]
    total = sum([x[1] for x in top2])
    tag_dist_h2[k] = {k:v/total for (k,v) in top2}

In [59]:
assert(list(tag_dist_h2.keys())==list(tag_dist_h2.keys()))

In [67]:
tag_dist_l2 = {}
for k, dist in list(tag_dist_l.items()):
    top2 = sorted(list(dist.items()),key= lambda x: x[1], reverse=True)[:2]
    total = sum([x[1] for x in top2])
    tag_dist_l2[k] = {k:v/total for (k,v) in top2}

In [68]:
list(tag_dist_l2.items())[:5]

[('स्वप्न', {'NOUN': 0.816, 'ADV': 0.184}),
 ('दाखवि', {'VERB': 1.0}),
 ('णे', {'VSUFF': 0.9328808446455505, 'NSUFF': 0.06711915535444947}),
 ('आणि', {'CCONJ': 0.9996929689898679, 'NSUFF': 0.00030703101013202335}),
 ('आश्वासन', {'ADV': 0.5871559633027523, 'NOUN': 0.41284403669724773})]